In [1]:
import pandas as pd
import networkx as nx
from datetime import datetime, timedelta, time
import time
import heapq
import hashlib
import math
import random
from collections import deque

data = pd.read_csv('..\connection_graph.csv', index_col=0)
data.head()
data.columns.values

C:\Users\User\AppData\Local\Temp\ipykernel_10892\1397015984.py:11: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('..\connection_graph.csv', index_col=0)


array(['company', 'line', 'departure_time', 'arrival_time', 'start_stop',
       'end_stop', 'start_stop_lat', 'start_stop_lon', 'end_stop_lat',
       'end_stop_lon'], dtype=object)

In [2]:
def normalize_and_parse_time(time_series):
    time_parts = time_series.str.split(':', expand=True).astype(int)
    hours, minutes, seconds = time_parts[0] % 24, time_parts[1], time_parts[2]
    normalized_times = hours.astype(str).str.zfill(2) + ':' + minutes.astype(str).str.zfill(2) + ':' + seconds.astype(str).str.zfill(2)
    return pd.to_datetime(normalized_times, format='%H:%M:%S').dt.time

data['departure_time'] = normalize_and_parse_time(data['departure_time'])
data['arrival_time'] = normalize_and_parse_time(data['arrival_time'])

# Vectorized hash generation
def vectorized_hash(lat_series, lon_series):
    geo_strings = lat_series.astype(str) + '_' + lon_series.astype(str)
    return geo_strings.apply(lambda x: hashlib.sha256(x.encode()).hexdigest())

data['start_stop_id'] = vectorized_hash(data['start_stop_lat'], data['start_stop_lon'])
data['end_stop_id'] = vectorized_hash(data['end_stop_lat'], data['end_stop_lon'])

data.head()

,company,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,start_stop_id,end_stop_id
0,MPK Autobusy,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna,51.148737,17.021069,51.147752,17.020539,c11c66389f5c0c02141b2bf412b286ea3a0b82bccdbbd8...,7f3a7f877a70f6e1434e0e53f7a060d2c93c89e8ee322f...
1,MPK Autobusy,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska),51.147752,17.020539,51.144385,17.023735,7f3a7f877a70f6e1434e0e53f7a060d2c93c89e8ee322f...,069b22abb9a894c7dcfe06e2215b67d660ffeec0ed71a0...
2,MPK Autobusy,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376,069b22abb9a894c7dcfe06e2215b67d660ffeec0ed71a0...,777750c42f1ec2a9aeec809295c2e4056da90554e22996...
3,MPK Autobusy,A,20:55:00,20:57:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.136632,17.030617,777750c42f1ec2a9aeec809295c2e4056da90554e22996...,4584214221503bb6ac00ca191b008a02950ad82a82a876...
4,MPK Autobusy,A,20:57:00,20:59:00,Bałtycka,Broniewskiego,51.136632,17.030617,51.135851,17.037383,4584214221503bb6ac00ca191b008a02950ad82a82a876...,8f1d1bd42d52557eeb64047ef07a1f439e5eb82a3cfd61...


In [3]:
graph = nx.MultiDiGraph()

for id, row in data.iterrows():
    graph.add_node(row['start_stop_id'], name=row['start_stop'], lat=row['start_stop_lat'], lon=row['start_stop_lon'])
    graph.add_node(row['end_stop_id'], name=row['end_stop'], lat=row['end_stop_lat'], lon=row['end_stop_lon'])

    edge_data = {
        'id': id,
        'company': row['company'],
        'line': row['line'],
        'departure_time': row['departure_time'],
        'arrival_time': row['arrival_time'],
    }
    graph.add_edge(row['start_stop_id'], row['end_stop_id'], **edge_data)

In [4]:
def time_difference(start, end):
    start_dt = datetime.combine(datetime.min, start)
    end_dt = datetime.combine(datetime.min, end)
    if end < start: # crossing midnight
        end_dt += timedelta(days=1)
    return (end_dt - start_dt).total_seconds() / 60 # minutes

In [5]:
def shortest_path_dijkstra(graph, start_stop_name, end_stop_name, start_time):
    calculation_start_time = time.time()
    start_nodes = [n for n, d in graph.nodes(data=True) if d['name'] == start_stop_name]
    end_nodes = [n for n, d in graph.nodes(data=True) if d['name'] == end_stop_name]
    pq = [(0, start_node, [], start_time) for start_node in start_nodes]  # cost, node, path, current_time

    visited = set()

    while pq:
        current_cost, current_node, path, current_time = heapq.heappop(pq)
    
        if current_node in visited:
            continue

        visited.add(current_node)

        if current_node in end_nodes:
            calculation_end_time = time.time()
            calculation_duration = calculation_end_time - calculation_start_time
            return path, current_cost, calculation_duration
        
        for next_node, edge_data in graph[current_node].items():
            for key, data in edge_data.items():
                departure_time = data['departure_time']
                arrival_time = data['arrival_time']
                
                if departure_time >= current_time:
                    waiting_time = time_difference(current_time, departure_time)
                    travel_time = time_difference(departure_time, arrival_time)
                    total_cost = current_cost + waiting_time + travel_time
                    heapq.heappush(pq, (total_cost, next_node, path + [data['id']], arrival_time))

    return [], float('inf') 

In [6]:
def print_path(path, data):
    if not path:
        print("No path found.")
        return
    
    last_line = None
    start_stop = None
    start_time = None
    
    for edge_id in path:
        edge_data = data.loc[edge_id]
        
        # Jeśli rozpoczynamy nową linię lub to pierwszy segment ścieżki
        if last_line != edge_data['line']:
            # Jeśli nie jest to pierwszy segment, zakończ poprzednią linię
            if last_line is not None:
                print(f"Line: {last_line}, from {start_stop} at {start_time} to {edge_data['start_stop']} at {edge_data['departure_time']}")
                print()  # Dodajemy pustą linię dla lepszej czytelności
                
            # Zaktualizuj dane dla nowej linii
            last_line = edge_data['line']
            start_stop = edge_data['start_stop']
            start_time = edge_data['departure_time']
    
    # Dla ostatniej linii na liście
    if last_line is not None:
        # Znajdź ostatni przystanek i czas przyjazdu na koniec ścieżki
        last_edge_data = data.loc[path[-1]]
        print(f"Line: {last_line}, from {start_stop} at {start_time} to {last_edge_data['end_stop']} at {last_edge_data['arrival_time']}")


In [7]:
def parse_time(time_str):
    return datetime.strptime(time_str, '%H:%M:%S').time()

In [8]:
path, cost, duration = shortest_path_dijkstra(graph, 'Ogród Botaniczny', 'Smolecka', parse_time('23:55:00'))

print_path(path, data)
print(f"Cost: {cost / 60} minutes")
print(f"Calculation duration: {duration} seconds")

Line: 2, from Ogród Botaniczny at 23:56:00 to Jedności Narodowej at 00:14:00

Line: 251, from Jedności Narodowej at 00:14:00 to Nowowiejska at 00:23:00

Line: 240, from Nowowiejska at 00:23:00 to DWORZEC GŁÓWNY at 00:38:00

Line: 206, from DWORZEC GŁÓWNY at 00:38:00 to DWORZEC AUTOBUSOWY at 00:46:00

Line: 240, from DWORZEC AUTOBUSOWY at 00:46:00 to Arkady (Capitol) at 00:53:00

Line: 248, from Arkady (Capitol) at 00:53:00 to Renoma at 00:56:00

Line: 206, from Renoma at 00:56:00 to Smolecka at 01:01:00
Cost: 1.1 minutes
Calculation duration: 0.6110439300537109 seconds


In [9]:
def heuristic(node, goal, graph):
    return math.sqrt((graph.nodes[node]['lat'] - graph.nodes[goal]['lat'])**2 + (graph.nodes[node]['lon'] - graph.nodes[goal]['lon'])**2) # Euclidean distance

def shortest_path_a_star(graph, start_stop_name, end_stop_name, start_time, optimization_criteria='t'):
    calculation_start_time = time.time()
    start_nodes = [n for n, d in graph.nodes(data=True) if d['name'] == start_stop_name]
    end_nodes = [n for n, d in graph.nodes(data=True) if d['name'] == end_stop_name]
    
    pq = []  # Initialize the priority queue
    
    for start_node in start_nodes:
        for end_node in end_nodes:
            heapq.heappush(pq, (0 + heuristic(start_node, end_node, graph), start_node, [], start_time, 0, None))  # cost, node, path, current_time, g(n), current_line

    visited = set()

    while pq:
        _, current_node, path, current_time, g_n, current_line = heapq.heappop(pq)
        if current_node in visited:
            continue
        visited.add(current_node)

        if current_node in end_nodes:
            calculation_end_time = time.time()
            calculation_duration = calculation_end_time - calculation_start_time
            return path, g_n, calculation_duration
        
        for next_node, edge_data in graph[current_node].items():
            for _, data in edge_data.items():
                departure_time = data['departure_time']
                arrival_time = data['arrival_time']
                if departure_time >= current_time:
                    if(optimization_criteria == 't'):
                        waiting_time = time_difference(current_time, departure_time)
                        travel_time = time_difference(departure_time, arrival_time)
                        g_n_next = g_n + waiting_time + travel_time
                    elif optimization_criteria == 'p':
                        if(data['line'] != current_line and current_line != None):
                            g_n_next = g_n + 1
                        else:
                            g_n_next = g_n
                    f_n_next = g_n_next + heuristic(next_node, end_node, graph)
                    heapq.heappush(pq, (f_n_next, next_node, path + [data['id']], arrival_time, g_n_next, data['line']))

    calculation_duration = calculation_end_time - calculation_start_time
    return [], float('inf'), calculation_duration

In [10]:
path, cost, duration = shortest_path_a_star(graph, 'Ogród Botaniczny', 'Smolecka', parse_time('23:55:55'), 'p')

print_path(path, data)
print(f"Cost: {cost} transfers")
print(f"Calculation duration: {duration} seconds")


Line: 2, from Ogród Botaniczny at 23:56:00 to Jedności Narodowej at 06:24:00

Line: 6, from Jedności Narodowej at 06:24:00 to Uniwersytet Wrocławski at 13:49:00

Line: 12, from Uniwersytet Wrocławski at 13:49:00 to pl. Strzegomski (Muzeum Współczesne) at 15:33:00

Line: 13, from pl. Strzegomski (Muzeum Współczesne) at 15:33:00 to Dolmed at 15:39:00

Line: 20, from Dolmed at 15:39:00 to Smolecka at 15:41:00
Cost: 4 transfers
Calculation duration: 1.7279846668243408 seconds


In [11]:
path, cost, duration = shortest_path_a_star(graph, 'Ogród Botaniczny', 'Smolecka', parse_time('23:55:00'), 't')

print_path(path, data)
print(f"Cost: {cost} minutes")
print(f"Calculation duration: {duration} seconds")

Line: 2, from Ogród Botaniczny at 23:56:00 to Jedności Narodowej at 00:14:00

Line: 251, from Jedności Narodowej at 00:14:00 to Nowowiejska at 00:23:00

Line: 240, from Nowowiejska at 00:23:00 to DWORZEC GŁÓWNY at 00:38:00

Line: 206, from DWORZEC GŁÓWNY at 00:38:00 to DWORZEC AUTOBUSOWY at 00:46:00

Line: 240, from DWORZEC AUTOBUSOWY at 00:46:00 to Arkady (Capitol) at 00:53:00

Line: 248, from Arkady (Capitol) at 00:53:00 to Renoma at 00:56:00

Line: 206, from Renoma at 00:56:00 to Smolecka at 01:01:00
Cost: 66.0 minutes
Calculation duration: 0.6540372371673584 seconds


## Tabu search



In [44]:
def avg_euclidian_distance(graph, start_stop_name, end_stop_name):
    start_nodes = [n for n, d in graph.nodes(data=True) if d['name'] == start_stop_name]
    end_nodes = [n for n, d in graph.nodes(data=True) if d['name'] == end_stop_name]
    
    distances = []
    for start_node in start_nodes:
        for end_node in end_nodes:
            distances.append(heuristic(start_node, end_node, graph))
    
    return sum(distances) / len(distances)

def calculate_shortest_distance_between_stops(graph, stops_list):
    distance_info = {}
    for i, start_stop in enumerate(stops_list):
        for end_stop in stops_list[i+1:]:
            h = avg_euclidian_distance(graph, start_stop, end_stop)
            distance_info[(start_stop, end_stop)] = h
            distance_info[(end_stop, start_stop)] = h
    return distance_info

def add_minutes_to_time(time, minutes):
    return (datetime.combine(datetime.min, time) + timedelta(minutes=minutes)).time()

def calculate_total_cost_and_path_for_solution(start_stop, solution, graph, optimization_criteria, start_time):
    total_cost = 0
    total_path = []
    current_stop = start_stop
    current_time = start_time
    for stop in solution:
        path, cost, _ = shortest_path_a_star(graph, current_stop, stop, current_time, optimization_criteria)
        total_cost += cost
        current_stop = stop
        total_path.extend(path)
        current_time = add_minutes_to_time(current_time, cost)
    
    # Add the path from the last stop to the start stop
    path, cost, _ = shortest_path_a_star(graph, solution[-1], start_stop, current_time, optimization_criteria)
    total_cost += cost
    total_path.extend(path)
    
    return total_cost, total_path

def generate_neighbors(current_solution):
    neighbors = []
    n = len(current_solution)
    for _ in range(10):  # Generate 10 neighbors for simplicity; adjust as needed
        new_solution = current_solution.copy()
        if random.choice([True, False]):
            # Swap two stops
            i, j = random.sample(range(n), 2)
            new_solution[i], new_solution[j] = new_solution[j], new_solution[i]
        else:
            # Reverse a segment
            i, j = sorted(random.sample(range(n), 2))
            new_solution[i:j+1] = reversed(new_solution[i:j+1])
        neighbors.append(new_solution)
    return neighbors

def get_full_path(start_stop, solution, paths_info):
    full_path = []
    current_stop = start_stop
    for stop in solution:
        path = paths_info[(current_stop, stop)]['path']
        full_path.extend(path)
        current_stop = stop

    full_path.extend(paths_info[(start_stop, solution[-1])]['path'])

    return full_path

def tabu_search(graph, start_stop_name, stops_list, optimization_criteria, start_time, tabu_list_limit=True):
    calculation_start_time = time.time()

    distance_info = calculate_shortest_distance_between_stops(graph, stops_list + [start_stop_name])

    if tabu_list_limit:
        tabu_list = deque(maxlen=len(stops_list) // 2)  # Limit the Tabu list to half the number of stops
    else:
        tabu_list = deque()
    
    current_solution = stops_list
    current_cost, current_path = calculate_total_cost_and_path_for_solution(start_stop_name, current_solution, graph, optimization_criteria, start_time)

    best_solution = current_solution
    best_cost = current_cost
    best_path = current_path
    max_iterations = 100
    iterations = 0

    while iterations < max_iterations:
        iterations += 1
        neighbors = generate_neighbors(current_solution)
        for neighbor in neighbors:
            if neighbor in tabu_list:
                continue  # Skip neighbors in the Tabu list
            cost, path = calculate_total_cost_and_path_for_solution(start_stop_name, neighbor, graph, optimization_criteria, start_time)
            if cost < best_cost:
                best_solution = neighbor
                best_cost = cost
                best_path = path
                # Add the current solution to the Tabu list to avoid revisiting
                tabu_list.append(list(current_solution))  # Ensure a copy of the solution is added

        # Update the current solution if a better one is found
        if best_cost < current_cost:
            current_solution = best_solution
            current_cost = best_cost
            current_path = best_path
        else:
            # Break if no improvement
            break

    calculation_end_time = time.time()
    calculation_duration = calculation_end_time - calculation_start_time

    # full_best_path = get_full_path(start_stop_name, best_solution, paths_info)

    return best_path, best_cost, calculation_duration

In [45]:
path, cost, duration = tabu_search(graph, 'Ogród Botaniczny', ['BISKUPIN', 'Tramwajowa', 'Piastowska'], 't', parse_time('10:55:00'))

print_path(path, data)
print(f"\nCost: {cost} minutes")
print(f"Calculation duration: {duration} seconds")

Line: 19, from Ogród Botaniczny at 10:58:00 to ZOO at 11:09:00

Line: 10, from ZOO at 11:09:00 to BISKUPIN at 11:16:00

Line: 2, from BISKUPIN at 11:16:00 to Kliniki - Politechnika Wrocławska at 11:26:00

Line: 1, from Kliniki - Politechnika Wrocławska at 11:26:00 to Piastowska at 11:34:00

Line: 19, from Piastowska at 11:34:00 to Ogród Botaniczny at 11:38:00
Cost: 43.0 minutes
Calculation duration: 15.441948652267456 seconds


In [46]:
path, cost, duration = tabu_search(graph, 'Ogród Botaniczny', ['BISKUPIN', 'Tramwajowa', 'Piastowska'], 't', parse_time('10:55:00'), False)

print_path(path, data)
print(f"\nCost: {cost} minutes")
print(f"Calculation duration: {duration} seconds")

Line: 19, from Ogród Botaniczny at 10:58:00 to ZOO at 11:09:00

Line: 10, from ZOO at 11:09:00 to BISKUPIN at 11:16:00

Line: 2, from BISKUPIN at 11:16:00 to Kliniki - Politechnika Wrocławska at 11:26:00

Line: 1, from Kliniki - Politechnika Wrocławska at 11:26:00 to Piastowska at 11:34:00

Line: 19, from Piastowska at 11:34:00 to Ogród Botaniczny at 11:38:00

Cost: 43.0 minutes
Calculation duration: 19.87299108505249 seconds
